### 0. đang chia file chạy cái pertured data
    1. thay từ mới vào tạo ra pertured data (masked_data_text folder)
    1. Lấy data tạo ra file json chạy model
### 1. Tạo masked word và lưu giống data cũ (cal_metric.py)
### 2. Lấy ra vector logit 
    1. convert interim -> txt 
    2. transform, prepare -> json 
    3. ghép label (từ 2.2) của câu gốc với câu mask
    4. đưa json (masked data 0.1) vào model lấy logit, prediction 
### 3. lấy được index của chỗ arg bị thay đổi của dự đoán sau khi masked data 
### 4. Tính inf 

## Step 0

In [27]:
import csv
import json
import os
import pandas as pd
import torch
from tqdm import tqdm
from mlm_utils.model_utils import TOKENIZER
from mlm_utils.transform_func import get_files

def read_data(readPath):
    
    with open(readPath, 'r', encoding = 'utf-8') as file:
        taskData = []
        for i, line in enumerate(file):
            sample = json.loads(line)
            taskData.append(sample)
            
    return taskData

def generate_data_and_write(dataDir, wriDir):
    
    def gen_data_line(line):
        
        origin_sen = TOKENIZER.decode(torch.tensor(line['origin_id']), skip_special_tokens = True)

        new_word_list_set = set([line['cos_neg'], line['cos_0'], line['cos_module_neg'], line['cos_module_0']])

        masked_data_set = list(map(lambda x: origin_sen.replace(line['masked_word'], x, 1), new_word_list_set))
        return line['origin_id'], masked_data_set
    
    files = get_files(dataDir)
    
    for file in tqdm(files):
        
        data = read_data(dataDir + file)
        
        masked_data_file = list(map(lambda x: gen_data_line(x), data))
       
        # Create a DataFrame
        df = pd.DataFrame(masked_data_file, columns=['origin_id', 'pertured_text'])
        df['index'] = df.index

        # Reorder columns
        df = df[['index', 'origin_id', 'pertured_text']]
        write_file = "pertured_data_{}".format(file.replace('.json', '.csv'))
        df.to_csv(os.path.join(wriDir, write_file), index=False, header=True)
        

In [28]:
generate_data_and_write(dataDir = 'pertured_data/', wriDir = 'masked_data_text/')

100%|██████████| 35/35 [00:11<00:00,  3.03it/s]


## Step 2

In [ ]:
# 2.1 Done
# convert csv to coNLL format and write to txt file
!python ../data_transformations.py --transform_file '/mnt/c/Users/Phat Pham/Documents/THESIS/SRLPredictionEasel/models/transform_file_mlm.yml'
# output file MLM/coNLL_txt

In [ ]:
# 2.2 Done
!python ../data_transformations.py --transform_file '/mnt/c/Users/Phat Pham/Documents/THESIS/SRLPredictionEasel/models/transform_file_conll.yml'
# nhớ chỉnh lại đường dẫn trong file transform_file_conll.yml

In [30]:
# 2.2 Done
# !python ../data_preparation.py --task_file tasks_file_SRL.yml --data_dir ../data/coNLL_tsv --max_seq_len 50

!python ../data_preparation.py \
        --task_file "/mnt/c/Users/Phat Pham/Documents/THESIS/SRLPredictionEasel/models/tasks_file_SRL.yml" \
        --data_dir ../MLM/coNLL_tsv_json \
        --max_seq_len 50

2024-05-09 16:44:23.464337: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-05-09 16:44:23.511364: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-09 16:44:24.160632: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/home/phatpham/anaconda3/envs/min_ds-env/lib/python3.8/site-packages/torch/cuda/__init__.py:546: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
2024-05-09 16:44:25.273226: E tensorflow/compiler/xla/stream_executor/cuda/cu